# Setup

In [1]:
from dotenv import load_dotenv
env = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_openai import ChatOpenAI
from IPython.display import Image
import IPython

/Users/robertwalling/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Tools

In [3]:
from typing import TypedDict, Annotated, Dict, Any
import operator

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [16]:
# Download PDF tool
from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.pydantic_v1 import BaseModel, Field
from bs4 import BeautifulSoup
import tiktoken
import requests

# Search for relevant menu
search_tool = TavilySearchResults()


# Scrape menu data
class ScrapeInput(BaseModel):
    url: str = Field(description="the URL of the menu page")

@tool("scrape_pdf", args_schema=ScrapeInput, return_direct=True)
def scrape_pdf(url: str):
    """Scrape a webpage that may include links to a restaurants current menu and return the links"""
    pdf_links = []
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        pdf_links = [link.get('href') for link in links if link.get('href').endswith('.pdf')]
    except Exception as e:
        print(f"failed to scrape {url} ERROR: {e}")
    return pdf_links

@tool("scrape_text", args_schema=ScrapeInput, return_direct=True)
def scrape_text(url: str):
    """Scrape the text directly from a website"""
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
        # Get text
        text = soup.get_text(separator=' ')
        # Break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # Break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # Drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
    except Exception as e:
        print(f"failed to scrape {url} ERROR: {e}")
        return None
    return text

# Encode & Upsert menu data
enc = tiktoken.encoding_for_model("gpt-4o")

class tokenizerInput(BaseModel):
    pdf_url: str = Field(description="The url of the PDF file that must be tokenized")

"""
# Tokenize the pdf 
@tool("tokenize-pdf", args_schema=tokenizerInput, return_direct=True)
def tokenize_pdf():
    pass
"""

'\n# Tokenize the pdf \n@tool("tokenize-pdf", args_schema=tokenizerInput, return_direct=True)\ndef tokenize_pdf():\n    pass\n'

In [17]:
tools = [search_tool, scrape_pdf, scrape_text]

# RAG Model

In [18]:
class Model:
    # Define the model
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.openai_inference)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    # Check if llm requires action 
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    # Run a tool ordered by the model
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}
    
    # Invokes the current message chain
    def openai_inference(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

In [19]:
prompt = """You are tasked with finding and storing the text content of a given restaurants Menu. 
The menu may be a PDF file that you will be required to scrape, or it will be text on a menu page. 
ONLY scrape menu data, this includes foods that the restaurant offers and DOES NOT include information about the restaurant itself or the menu iteself.
Use the search tool to find the url of the menu, if you cannot find the pdf url directly use the scrape_pdf tool to scrape webpages for the url.
If the menu isn't saved as a PDF, and is instead saved as text on a webpage, use the scrape_text tool to gather the text from the page.
DO NOT attempt to download blank menus. Save the file using the PDF download tool, name the PDF after the restaurant.
Once you have downloaded the PDF there are no more tasks to be done. 
"""

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
)

model = Model(llm, tools, system=prompt)

In [20]:
messages = ["CRISP Rochester"]
result = model.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'CRISP Rochester menu'}, 'id': 'call_HXfS5iQGmPo9WPH0T7QvXSvA', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'scrape_pdf', 'args': {'url': 'https://crisprochester.com/menus/'}, 'id': 'call_TZTEz2dmswggrm36EFMeLURz', 'type': 'tool_call'}
Calling: {'name': 'scrape_pdf', 'args': {'url': 'https://crisprochester.com/wp-content/uploads/Lunch-and-Dinner.pdf'}, 'id': 'call_y2x4hbhHrvZ9U1vcGIL2mHyr', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'CRISP Rochester menu PDF'}, 'id': 'call_h4Svl59jiOnk3RSargFx9WkB', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'download_pdf', 'args': {'file_url': 'https://crisprochester.com/wp-content/uploads/Lunch-and-Dinner.pdf', 'file_name': 'CRISP_Rochester_Lunch_and_Dinner'}, 'id': 'call_aXzqqPR8Ol5Kggx7aXsi0zWc', 'type': 'tool_call'}


KeyError: 'download_pdf'

In [ ]:
result

In [17]:
import pygraphviz
Image(model.graph.get_graph().draw_png())

ModuleNotFoundError: No module named 'pygraphviz'

# Datastore

In [ ]:
# Tokenize the pdf 
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4o")


In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone()

pc.create_index(
    name="quickstart",
    dimension=5120, # Lamma 2 embedding dim
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)